In [16]:
import os
import pandas as pd
import git
import numpy as np



# folder = "C:\\Users\\Van\\Desktop\\vancanwin\\pgap3cdg\\01.20.23\\"

def csv_to_excel(folder,filename):
    filetype = filename.split(".")[-1]
    if "csv" in filetype:
        df = pd.read_csv(folder+filename, sep='delimiter',on_bad_lines='skip')
        filename = filename.split(".")[0]+".xlsx"
        df.to_excel(folder+filename)
    
def normalize_plate384(folder,filename):
    '''Enter file to read plate and normalize data'''
    print(folder+filename)
    filetype = filename.split(".")[-1]
    df = pd.DataFrame([])
    if "csv" not in filetype:
        df = pd.read_excel(folder+filename)
        filename = filename.split(".")[0]+".csv"
        df.to_csv (folder+filename, index = None, header=True)
    
    df = pd.DataFrame([])
    df = pd.read_csv(folder+filename, encoding = "utf-8",sep='delimiter')
    df = df.iloc[:,0].str.split(",",expand=True)



    df_results_index = df[df.iloc[:,0].str.contains('Results', na=False)].index
    for i, index in enumerate(df_results_index):
        dateset_name = df.iloc[index,0]
        new_header = df.iloc[index+1,1:]
        df_results = df.iloc[index+2:index+18,1:25] #row,col
        df_results = df_results.replace("",0)
        print(df_results)
        df_results = df_results.astype(int)
        df_results.columns = new_header
        df_median_neg_controls = int(df_results.iloc[:,0:2].stack().median())
        df_min = df_results.stack().min()
        df_max = df_results.stack().max()
        df_range = df_max-df_min
        df_results_norm = (df_results - df_min)/df_range
        df.iloc[index+2:index+18,1:] = df_results_norm
        print("processed...",i)
    df.to_csv(folder+"\\completed\\"+filename.split(".")[0]+"_normalized."+filename.split(".")[-1])
    return(df)

def zscore_plate384(folder,filename):
    '''Enter file to read plate and normalize data'''
    print(folder+filename)
    filetype = filename.split(".")[-1]
    df = pd.DataFrame([])
    if "csv" in filetype:
        df = pd.read_csv(folder+filename, sep='delimiter',on_bad_lines='skip')
        df = df.iloc[:,0].str.split(",",expand=True)
    else:
        df = pd.read_csv(folder+filename)
        
    df_initial = df
    df_results_index = df[df.iloc[:,0].str.contains('Results', na=False)].index
    normalized_dataframe = pd.DataFrame([])
    # print("DF",df, "INDEX",df_results_index[0])
    for i, index in enumerate(df_results_index):
        dateset_name = df.iloc[index,0]
        new_header = df.iloc[index+1,1:]
        df_results = df.iloc[index+2:index+18,1:] #row,col
        # print("i and index",i,index, dateset_name, df_results)
        df_results = df_results.replace("",0)
        df_results = df_results.astype(int)

        df_results.columns = new_header
        df_min = df_results.stack().min()
        df_max = df_results.stack().max()
        df_range = df_max-df_min
        df_results_norm = (df_results - df_min)/df_range
        
        df_mean = df_results_norm.stack().mean()
        df_std = df_results_norm.stack().std()
        df_results_zscore = (df_results_norm - df_mean)/df_std
        df.iloc[index+2:index+18,1:] = df_results_zscore
    df.to_csv(folder+"\\completed\\"+filename.split(".")[0]+"_zscore."+filename.split(".")[-1])

    return(df)


repo = git.Repo('.', search_parent_directories=True)
folder = repo.working_tree_dir +"\\yeast-data\\"

plate_list = os.listdir(folder)
print(plate_list,folder)

for file in plate_list:
    if "plate-data" in file:
        csv_to_excel(folder,file)
        normalize_plate384(folder,file)
        zscore_plate384(folder,file)

['archive', 'completed', 'plate-data-20230307_Perlara PIGA_CDG.csv', 'plate-data-20230307_Perlara PIGA_CDG.xlsx'] C:\Users\vandu\Desktop\vancanwin\yeast-data\
C:\Users\vandu\Desktop\vancanwin\yeast-data\plate-data-20230307_Perlara PIGA_CDG.csv
        1       2       3       4       5       6       7       8       9   \
36  632840  582400  581320  595760  458560  605440  620000  572200  663760   
37  640600  616680   48200  570360  580800  645360  540480  669360  502920   
38  645240  603040  627440  618960  628440  635280  565680   73720  604080   
39  676520  601840  622920  619880  645200  638000  129760  660160  620400   
40  657400  621640  640640  562640  640720  622080  550760  656400  697640   
41  696000  631000  618640  648600  115320  626040   52960  636760  254520   
42  696440  646520  615880  670400   43080  592040  664080  548400  631920   
43  680320  642000  626000  592840  689520  494160  537520  666320  700480   
44  669280  643360  656080  665440   50400  114320  65

C:\Users\vandu\AppData\Local\Temp\ipykernel_2736\965978282.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(folder+filename, sep='delimiter',on_bad_lines='skip')
C:\Users\vandu\AppData\Local\Temp\ipykernel_2736\965978282.py:28: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(folder+filename, encoding = "utf-8",sep='delimiter')


ValueError: Length mismatch: Expected axis has 24 elements, new values have 30 elements